In [1]:
import pandas as pd

# answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
#                                                                  'recommend','table','picture','video','numword',
#                                                                  'thumbsup','thumbsdown','comment','collect','thank',
#                                                                  'report','nohelp','oppose'])
invite_info=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
invite_info_evaluate=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

In [2]:
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])

In [3]:
member_info.drop_duplicates(['uid'],inplace=True)
ques_info.drop_duplicates(['qid'],inplace=True)

In [4]:
train=invite_info[['qid','uid','target']]
test=invite_info_evaluate[['qid','uid']]

In [5]:
member_info=member_info[['uid','topic_n']]
ques_info=ques_info[['qid','qtopic']]

In [6]:
train=pd.merge(train,member_info,on='uid',how='left')
test=pd.merge(test,member_info,on='uid',how='left')

train=pd.merge(train,ques_info,on='qid',how='left')
test=pd.merge(test,ques_info,on='qid',how='left')

In [7]:
train.head()

,qid,uid,target,topic_n,qtopic
0,Q2166419046,M401693808,0,"T2794:1.1527717,T9701:0.94830835,T9533:0.85710...","T456,T112,T9566,T5310"
1,Q1550017551,M3392373099,0,"T1470:1.4682752,T235:1.4412646,T4692:1.3516154...","T2,T3095"
2,Q604029601,M2317670257,0,"T13383:1.4091244,T1955:1.385072,T8943:1.314409...","T6090,T2156,T97,T456"
3,Q2350061229,M1618461867,0,"T671:1.4648209,T3772:1.2451016,T974:1.2110636,...",T856
4,Q2443223942,M3544409350,0,"T4876:1.1308655,T2467:0.9645766,T245:0.9475922...","T26,T76,T17"


In [8]:
train['topic_n'] = train['topic_n'].astype(str).map(lambda x:x.split(','))
train['qtopic_len'] = train['topic_n'].map(lambda x:len(x))

In [9]:
test['topic_n'] = test['topic_n'].astype(str).map(lambda x:x.split(','))
test['qtopic_len'] = test['topic_n'].map(lambda x:len(x))

In [10]:
from tqdm import tqdm

def flatten_topic(df):    
    u = []
    a = []
    q=[]
    for i in tqdm(range(len(df['topic_n'].values))):
        u += [df['uid'].values[i]]*df['qtopic_len'].values[i]
        q += [df['qid'].values[i]]*df['qtopic_len'].values[i]
        a += list(df['topic_n'].values[i])
        
    new_df = pd.DataFrame()
    new_df['uid'] = u
    new_df['qid'] = q
    new_df['topic_n'] = a
        
    return new_df

In [11]:
deal_train=flatten_topic(train)
deal_test=flatten_topic(test)

100%|██████████| 1141718/1141718 [00:25<00:00, 45202.13it/s]


In [12]:
from tqdm import tqdm
tqdm.pandas()

In [13]:
deal_train['topic_n_']=deal_train['topic_n'].progress_apply(lambda x:x.split(':'))

100%|██████████| 75417893/75417893 [04:28<00:00, 280677.33it/s]


In [14]:
deal_test['topic_n_']=deal_test['topic_n'].progress_apply(lambda x:x.split(':'))

100%|██████████| 8726071/8726071 [00:24<00:00, 356224.93it/s]


In [15]:
topic_n_tn=pd.DataFrame(deal_train['topic_n_'].tolist())
topic_n_te=pd.DataFrame(deal_test['topic_n_'].tolist())

In [16]:
deal_train=pd.concat([deal_train,topic_n_tn],axis=1)
deal_test=pd.concat([deal_test,topic_n_te],axis=1)
deal_train['id']=deal_train['uid']+deal_train['qid']
deal_test['id']=deal_test['uid']+deal_test['qid']

In [17]:
all_data=pd.concat([deal_train,deal_test])

In [18]:
all_data=all_data.fillna(0)

In [19]:
import numpy as np

In [20]:
all_data[1]=all_data[1].astype(float).replace([np.inf,-np.inf],0)

In [21]:
all_data.head()

,uid,qid,topic_n,topic_n_,0,1,id
0,M401693808,Q2166419046,T2794:1.1527717,"[T2794, 1.1527717]",T2794,1.152772,M401693808Q2166419046
1,M401693808,Q2166419046,T9701:0.94830835,"[T9701, 0.94830835]",T9701,0.948308,M401693808Q2166419046
2,M401693808,Q2166419046,T9533:0.8571003,"[T9533, 0.8571003]",T9533,0.857100,M401693808Q2166419046
3,M401693808,Q2166419046,T2396:0.8536834,"[T2396, 0.8536834]",T2396,0.853683,M401693808Q2166419046
4,M401693808,Q2166419046,T11120:0.8136603,"[T11120, 0.8136603]",T11120,0.813660,M401693808Q2166419046


In [22]:
id_=all_data[['id']]
len(id_)

84143964

In [23]:
len(train)+len(test)

10630880

In [24]:
x=pd.concat([train,test])
x['id']=x['uid']+x['qid']
#x=x.drop_duplicates(['id'])
len(x.drop_duplicates(['id']))

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


10628231

In [25]:
len(x)

10630880

In [26]:
from tqdm import tqdm
cat_col=[0,'id']
from sklearn.preprocessing import LabelEncoder
for feat in tqdm(cat_col):
    lbl = LabelEncoder()
    all_data[feat] = lbl.fit_transform(all_data[feat].astype('str'))

100%|██████████| 2/2 [03:30<00:00, 95.00s/it]


In [27]:
all_data=all_data.sort_values('id')

In [28]:
x=all_data

In [29]:
x['id_']=x['uid']+x['qid']

In [30]:
len(x)

84143964

In [31]:
x=x.drop_duplicates(['id_'])

In [32]:
len(x)

10628231

In [33]:
y=pd.concat([train,test])
y['id_']=y['uid']+y['qid']
y=y.drop_duplicates(['id_'])

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [34]:
y=y[['qtopic','target','id_']]

In [35]:
x=x.reset_index(drop=True)

In [36]:
x=pd.merge(x,y,how='left',on='id_')

In [37]:
len(all_data)

84143964

In [38]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
all_data=reduce_mem_usage(all_data)

Mem. usage decreased to 4493.77 Mb (22.2% reduction)


In [39]:
from scipy.sparse import csr_matrix

csr_= csr_matrix((all_data[1], (all_data['id'], all_data[0])))#构造csr方法


In [40]:
csr_.shape

(10628231, 22872)

In [41]:
x['qtopic']=x['qtopic'].progress_apply(lambda x:' '.join(str(x).split(',')))

100%|██████████| 10628231/10628231 [00:40<00:00, 261622.13it/s]


In [42]:
import numpy as np
#import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import gc
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

In [43]:
tfidf = CountVectorizer(ngram_range=(1,1))
tf_qtopic = tfidf.fit_transform(x['qtopic'])

In [44]:
from scipy import sparse

csr_all=sparse.csc_matrix(sparse.hstack((tf_qtopic, csr_)))

In [45]:
csr_all.shape

(10628231, 78143)

In [46]:
index_=np.array(x[x['target'].notnull()].reset_index()['index'])

In [47]:
index_t=np.array(x[x['target'].isnull()].reset_index()['index'])

In [48]:
train_feature = csr_[index_]
test_feature = csr_[index_t]

In [49]:
print(train_feature.shape,test_feature.shape)

(9486821, 22872) (1141410, 22872)


In [50]:
x.head()

,uid,qid,topic_n,topic_n_,0,1,id,id_,qtopic,target
0,M1000000382,Q1282394327,T6:0.7224611,"[T6, 0.7224611]",18800,0.722461,0,M1000000382Q1282394327,T547 T89,0.0
1,M1000000382,Q2418257178,T150:0.7708566,"[T150, 0.7708566]",4403,0.770857,1,M1000000382Q2418257178,T89,0.0
2,M1000000382,Q2557202265,T101:0.6198262,"[T101, 0.6198262]",95,0.619826,2,M1000000382Q2557202265,T476,0.0
3,M1000000382,Q2574396618,T9:1.0592799,"[T9, 1.0592799]",21899,1.059280,3,M1000000382Q2574396618,T13 T567,1.0
4,M1000000382,Q3739876694,T93:0.56995815,"[T93, 0.56995815]",22198,0.569958,4,M1000000382Q3739876694,T724 T89,0.0


In [51]:
train_id=x.iloc[index_,0:2]
test_id=x.iloc[index_t,0:2]

In [52]:
print(train_id.shape,test_id.shape)

(9486821, 2) (1141410, 2)


In [53]:
target=x.iloc[index_,-1].reset_index(drop=True)

In [54]:
def sgdstacking():
    gc.collect()
    print('sgd stacking')
    df_stack=pd.DataFrame()
    n_folds=5
    num_class=2
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        sgd = SGDClassifier(random_state=1017,
                            verbose=1,
                            loss='log',
                            penalty='elasticnet',
                            shuffle=True,
                            n_jobs=20,
                            l1_ratio=0.0001,
                            alpha=1e-05,
                            class_weight=None)
        gc.collect()
        sgd.fit(train_feature[tr], target[tr])
        score_va = sgd.predict_proba(train_feature[va])
        score_te = sgd.predict_proba(test_feature)
        stack_train[va] = score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_sgd_{}'.format(i)] = stack[:, i]
    print('sgd done')
    return df_stack

In [55]:
sgdstacking=sgdstacking()

sgd stacking
stack:1/5


/root/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


-- Epoch 1
Norm: 14.64, NNZs: 22604, Bias: -1.095281, T: 7589456, Avg. loss: 0.521168
Total training time: 7.77 seconds.
-- Epoch 2
Norm: 13.33, NNZs: 22561, Bias: -1.094389, T: 15178912, Avg. loss: 0.460907
Total training time: 15.75 seconds.
-- Epoch 3
Norm: 13.00, NNZs: 22562, Bias: -1.092141, T: 22768368, Avg. loss: 0.459516
Total training time: 23.22 seconds.
-- Epoch 4
Norm: 12.85, NNZs: 22549, Bias: -1.095849, T: 30357824, Avg. loss: 0.458919
Total training time: 30.54 seconds.
-- Epoch 5
Norm: 12.76, NNZs: 22538, Bias: -1.098846, T: 37947280, Avg. loss: 0.458580
Total training time: 38.21 seconds.
stack:2/5
-- Epoch 1
Norm: 14.53, NNZs: 22578, Bias: -1.094728, T: 7589457, Avg. loss: 0.521409
Total training time: 6.51 seconds.
-- Epoch 2
Norm: 13.27, NNZs: 22553, Bias: -1.086954, T: 15178914, Avg. loss: 0.460952
Total training time: 12.21 seconds.
-- Epoch 3
Norm: 12.95, NNZs: 22564, Bias: -1.100146, T: 22768371, Avg. loss: 0.459567
Total training time: 18.35 seconds.
-- Epoch 4

In [56]:
def lrstacking():
    num_class=2
    df_stack=pd.DataFrame()
    n_folds=5
    gc.collect()
    print('LR stacking')
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (trn_idx, val_idx) in enumerate(folds.split(target)):
        print('stack:%d' % ((i + 1) ))
        clf = LogisticRegression(random_state=1017, C=8,solver='sag',verbose=1,n_jobs=-1)
        clf.fit(train_feature[trn_idx], target[trn_idx])
        score_va = clf.predict_proba(train_feature[val_idx])
        score_te = clf.predict_proba(test_feature)
        stack_train[val_idx, :] = score_va
        stack_test[:, :] += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfctn_lr_{}'.format(i)] = stack[:, i]
    print('lr done')
    return df_stack

In [57]:
lrstacking=lrstacking()

LR stacking
stack:1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 58 concurrent workers.


max_iter reached after 637 seconds


/root/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.6min finished


stack:2


...

stack:5


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 58 concurrent workers.


max_iter reached after 627 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.5min finished


lr done


In [58]:
def ridgestacking():
    gc.collect()
    print('Ridge stacking')
    df_stack=pd.DataFrame()
    num_class=2
    n_folds=5
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        ridge = RidgeClassifier(random_state=1017,solver='sag',)
        ridge.fit(train_feature[tr], target[tr])
        score_va = ridge._predict_proba_lr(train_feature[va])
        score_te = ridge._predict_proba_lr(test_feature)
        stack_train[va] += score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_ridge_{}'.format(i)] = stack[:, i]
    print('ridge done')
    return df_stack

In [59]:
ridgestacking=ridgestacking()

Ridge stacking
stack:1/5
stack:2/5
stack:3/5
stack:4/5
stack:5/5
ridge done


In [60]:
result=pd.concat([lrstacking,sgdstacking,ridgestacking],axis=1)
result

,tfctn_lr_0,tfctn_lr_1,tfc_sgd_0,tfc_sgd_1,tfc_ridge_0,tfc_ridge_1
0,0.773847,0.226153,0.781423,0.218577,0.633810,0.366190
1,0.772974,0.227026,0.793162,0.206838,0.633432,0.366568
2,0.772974,0.227026,0.793162,0.206838,0.633432,0.366568
3,0.772974,0.227026,0.793162,0.206838,0.633432,0.366568
4,0.772974,0.227026,0.793162,0.206838,0.633432,0.366568
5,0.772974,0.227026,0.793162,0.206838,0.633432,0.366568
6,0.772642,0.227358,0.789413,0.210587,0.633291,0.366709
7,0.772974,0.227026,0.793162,0.206838,0.633432,0.366568
8,0.938658,0.061342,0.916265,0.083735,0.717900,0.282100
9,0.933809,0.066191,0.910290,0.089710,0.712891,0.287109


In [61]:
len(result)

10628231

In [62]:
len(train_id)+len(test_id)

10628231

In [63]:
result.drop(['tfctn_lr_1','tfc_sgd_1','tfc_ridge_1'],axis=1,inplace=True)

In [64]:
id_=pd.concat([train_id,test_id])

In [65]:
result=pd.concat([id_,result],axis=1)

In [67]:
test=test.merge(result,on =['qid','uid'],how='left')

In [69]:
len(test)+len(train)

10630880

In [123]:
result.to_csv('feature_topic_n_stacking.csv',index=False)

In [ ]:
feature.to_hdf('feature_qid_freq.h5', key='data')